In [1]:
!pip install pandas numpy scikit-learn nltk sentence-transformers

In [2]:
!pip install PyPDF2 langchain

In [3]:
import PyPDF2
import re
import os
import pandas as pd # pandas is useful for organizing extracted data later

# --- Configuration ---
# Set the directory where your PDF books are located
# Make sure your PDF files are in a 'books' subfolder within your ChatBotProject folder.
pdf_dir = 'books'

# List of PDF file names you want to process
pdf_files = [
    'C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf',
    'C:/Users/Ayush Jindal/Downloads/Wallach.pdf',
    'C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf',
    "C:/Users/Ayush Jindal/Downloads/guidelines-on-mental-health-promotive-and-preventive-interventions-for-adolescents-hat_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/overweight-and-obesity-management-pdf-66143959958725_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/constipation-in-children-and-young-people-diagnosis-and-management-pdf-975757753285_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-in-adults-prevention-and-lifestyle-weight-management-programmes-pdf-75545293071301_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/fatty acid intake.pdf",
    "C:/Users/Ayush Jindal/Downloads/WHO guideline for physics activity.pdf",
    "C:/Users/Ayush Jindal/Downloads/physical-activity-exercise-referral-schemes-pdf-1996418406085_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/bipolar-disorder-assessment-and-management-pdf-35109814379461_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-clinical-assessment-and-management-pdf-75545363615173_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/IND301-20250415_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/hypertension.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICE 1.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICD guideline mental health.pdf",
    "C:/Users/Ayush Jindal/Downloads/adolescent-health.pdf",
    "C:/Users/Ayush Jindal/Downloads/B1.pdf",
    "C:/Users/Ayush Jindal/Downloads/Mandell, Douglas, and Bennetts Principles and Practice of Infectious Diseases 2 Vol. Set, Eighth Edition (John E. Bennett, Raphael Dolin etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Kumar and Clarks Clinical Medicine (Parveen Kumar, Michael L Clark) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Harrisons Principles of Internal Medicine, Twenty-First Edition (Vol.1  Vol.2) (Joseph Loscalzo, Anthony S. Fauci etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Robbins pathologic basis of disease. (Cotran, Ramzi S., 1932-,Saunders etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Rubin's Pathology Clinicopathologic Foundations of Medicine, 7th Edition.pdf",
    "C:/Users/Ayush Jindal/Downloads/Netters sports medicine (Madden Ch.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Oxford Handbook of General Practice ( etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/toaz.info-current-diagnosis-amp-treatment-psychiatrypdf-pr_3071492a6593c0055faf81e303800df3.pdf",
    "C:/Users/Ayush Jindal/Downloads/Grays Anatomy For Students 4th-edition.pdf",
    "C:/Users/Ayush Jindal/Downloads/Essentials of Musculoskeletal Care, 5th ed (April Armstrong, Mark C Hubbard etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Epidemiology (Leon Gordis) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/CURRENT Medical Diagnosis and Treatment 2021 Maxine A. Papadakis, Stephen J. McPhee, Michael W. Rabow, ( etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Cecil Essentials of Medicine (Edward J. Wing MD  FACP  FIDSA (editor) etc.) (Z-Library).pdf"
    
]

# --- Text Extraction Function ---
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file page by page.
    Includes error handling for missing files or extraction issues.
    """
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f"  Extracting text from {num_pages} pages...")
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                except Exception as page_e:
                    print(f"    Warning: Could not extract text from page {page_num + 1} of {os.path.basename(pdf_path)}: {page_e}")
                    text += "[PAGE_TEXT_EXTRACTION_ERROR]"
        print(f"Successfully extracted raw text from: {os.path.basename(pdf_path)}")
    except FileNotFoundError:
        print(f"Error: PDF file not found at '{pdf_path}'. Please check the path and filename.")
    except Exception as e:
        print(f"Error opening or reading PDF '{pdf_path}': {e}")
    return text

# --- Initial Text Cleaning Function ---
def clean_initial_pdf_text(text):
    """
    Performs initial cleaning on raw extracted PDF text.
    Removes common PDF artifacts and standardizes formatting.
    """
    # 1. Remove source tags like or 

    # 2. Remove common PDF rendering artifacts like (cid:xx)
    text = re.sub(r'\(cid:\d+\)', '', text)

    # 3. Remove specific repeating headers/footers based on observation of your PDFs
    text = re.sub(r'Routine Blood\n Results\n Explained\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Routine Blood Results Explained', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in\n Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'A practical guide\n to interpreting blood\n test results', '', text, flags=re.IGNORECASE)

    # 4. Remove page numbers. This is heuristic and targets various page number patterns.
    text = re.sub(r'^\s*\d+\s*(\n|$)', '\n', text, flags=re.MULTILINE)
    text = re.sub(r'(\n|^)\s*\d+\s*(\n|$)', '\n', text)
    text = re.sub(r'Page \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Chapter \d+\s*\d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^\s*\d{1,3}\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*(viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 5. Remove TOC-like entries that remain, or prefaces/abbreviations
    text = re.sub(r'^\s*(Contents|Preface|Abbreviations|Introduction|Part \d+|Chapter \d+).*?\b(?:vii|viii|ix|x|xi|xii|\d+)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 6. Standardize whitespace: Replace multiple whitespaces (including newlines) with single space
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'([.?!])\s*(?=[A-Z0-9]|$)', r'\1\n\n', text) # Add double newline after sentences
    text = re.sub(r'\n{3,}', '\n\n', text) # Reduce excessive newlines

    # 7. Remove hyphenation from words split across lines
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)

    # 8. Clean specific table artifacts from contents that might be extracted oddly
    text = re.sub(r'"([^"]+)"\s*,\s*"([^"]+)"', r'\1 \2', text)
    text = re.sub(r'Objectives and Scope\n', '', text)
    text = re.sub(r'Table \d+\.\d+:.*?\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'The following table:', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Mode LastWriteTime Length ---- ------------- ------', '', text)

    return text

# --- Main Processing Loop ---
cleaned_texts_by_book = {}

for pdf_file_name in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file_name)
    print(f"\n--- Processing: {pdf_file_name} ---")

    raw_text = extract_text_from_pdf(pdf_path)
    if raw_text:
        cleaned_text = clean_initial_pdf_text(raw_text)
        cleaned_texts_by_book[pdf_file_name] = cleaned_text
        print(f"Initial cleaning complete for {pdf_file_name}. Cleaned length: {len(cleaned_text)} characters.")
    else:
        print(f"Skipping {pdf_file_name} due to extraction error or empty content.")

print("\n--- Initial Text Cleaning Summary ---")
for book_name, text_content in cleaned_texts_by_book.items():
    print(f"Book: {book_name[:35]}... | Length: {len(text_content)} chars")
    print("Sample (first 1000 chars):\n", text_content[:1000])
    print("-" * 50)


--- Processing: C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf ---
  Extracting text from 172 pages...
Successfully extracted raw text from: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
Initial cleaning complete for C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Cleaned length: 264579 characters.

--- Processing: C:/Users/Ayush Jindal/Downloads/Wallach.pdf ---
  Extracting text from 1884 pages...
Successfully extracted raw text from: Wallach.pdf
Initial cleaning complete for C:/Users/Ayush Jindal/Downloads/Wallach.pdf. Cleaned length: 3245859 characters.

--- Processing: C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf ---
  Extracting text from 119 pages...
Successfully extracted raw text f

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import PyPDF2
import re
import os
import pandas as pd # pandas is useful for organizing extracted data later

# --- Configuration ---
# Set the directory where your PDF books are located
# Make sure your PDF files are in a 'books' subfolder within your ChatBotProject folder.
pdf_dir = 'books' # This assumes 'books' is a direct subdirectory of where your script runs.
                   # Given your full paths, it seems you're using absolute paths directly,
                   # so pdf_dir might not be strictly needed for joining if paths are absolute.
                   # However, if you move files into 'books', this setup is correct.

# List of PDF file names you want to process
pdf_files = [
    'C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf',
    'C:/Users/Ayush Jindal/Downloads/Wallach.pdf',
    'C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf',
    "C:/Users/Ayush Jindal/Downloads/guidelines-on-mental-health-promotive-and-preventive-interventions-for-adolescents-hat_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/overweight-and-obesity-management-pdf-66143959958725_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/constipation-in-children-and-young-people-diagnosis-and-management-pdf-975757753285_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-in-adults-prevention-and-lifestyle-weight-management-programmes-pdf-75545293071301_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/fatty acid intake.pdf",
    "C:/Users/Ayush Jindal/Downloads/WHO guideline for physics activity.pdf",
    "C:/Users/Ayush Jindal/Downloads/physical-activity-exercise-referral-schemes-pdf-1996418406085_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/bipolar-disorder-assessment-and-management-pdf-35109814379461_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-clinical-assessment-and-management-pdf-75545363615173_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/IND301-20250415_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/hypertension.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICE 1.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICD guideline mental health.pdf",
    "C:/Users/Ayush Jindal/Downloads/adolescent-health.pdf",
    "C:/Users/Ayush Jindal/Downloads/B1.pdf",
    "C:/Users/Ayush Jindal/Downloads/Mandell, Douglas, and Bennetts Principles and Practice of Infectious Diseases 2 Vol. Set, Eighth Edition (John E. Bennett, Raphael Dolin etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Kumar and Clarks Clinical Medicine (Parveen Kumar, Michael L Clark) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Harrisons Principles of Internal Medicine, Twenty-First Edition (Vol.1  Vol.2) (Joseph Loscalzo, Anthony S. Fauci etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Robbins pathologic basis of disease. (Cotran, Ramzi S., 1932-,Saunders etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Rubin's Pathology Clinicopathologic Foundations of Medicine, 7th Edition.pdf",
    "C:/Users/Ayush Jindal/Downloads/Netters sports medicine (Madden Ch.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Oxford Handbook of General Practice ( etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/toaz.info-current-diagnosis-amp-treatment-psychiatrypdf-pr_3071492a6593c0055faf81e303800df3.pdf",
    "C:/Users/Ayush Jindal/Downloads/Grays Anatomy For Students 4th-edition.pdf",
    "C:/Users/Ayush Jindal/Downloads/Essentials of Musculoskeletal Care, 5th ed (April Armstrong, Mark C Hubbard etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Epidemiology (Leon Gordis) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/CURRENT Medical Diagnosis and Treatment 2021 Maxine A. Papadakis, Stephen J. McPhee, Michael W. Rabow, ( etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Cecil Essentials of Medicine (Edward J. Wing MD  FACP  FIDSA (editor) etc.) (Z-Library).pdf"
]

# --- Text Extraction Function ---
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file page by page.
    Includes error handling for missing files or extraction issues.
    """
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f"  Extracting text from {num_pages} pages...")
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                except Exception as page_e:
                    print(f"    Warning: Could not extract text from page {page_num + 1} of {os.path.basename(pdf_path)}: {page_e}")
                    text += "[PAGE_TEXT_EXTRACTION_ERROR]"
        print(f"Successfully extracted raw text from: {os.path.basename(pdf_path)}")
    except FileNotFoundError:
        print(f"Error: PDF file not found at '{pdf_path}'. Please check the path and filename.")
    except Exception as e:
        print(f"Error opening or reading PDF '{pdf_path}': {e}")
    return text

# --- Initial Text Cleaning Function ---
def clean_initial_pdf_text(text):
    """
    Performs initial cleaning on raw extracted PDF text.
    Removes common PDF artifacts and standardizes formatting.
    """
    # 1. Remove common PDF rendering artifacts like (cid:xx)
    text = re.sub(r'\(cid:\d+\)', '', text)

    # 2. Remove specific repeating headers/footers based on observation of your PDFs
    text = re.sub(r'Routine Blood\n Results\n Explained\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Routine Blood Results Explained', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in\n Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'A practical guide\n to interpreting blood\n test results', '', text, flags=re.IGNORECASE)

    # 3. Remove page numbers. This is heuristic and targets various page number patterns.
    # Note: Some of these regex might be too aggressive and remove legitimate numbers.
    # We will refine page number and header/footer removal in the advanced phase.
    text = re.sub(r'^\s*\d+\s*(\n|$)', '\n', text, flags=re.MULTILINE)
    text = re.sub(r'(\n|^)\s*\d+\s*(\n|$)', '\n', text)
    text = re.sub(r'Page \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Chapter \d+\s*\d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^\s*\d{1,3}\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*(viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 4. Remove TOC-like entries that remain, or prefaces/abbreviations
    text = re.sub(r'^\s*(Contents|Preface|Abbreviations|Introduction|Part \d+|Chapter \d+).*?\b(?:vii|viii|ix|x|xi|xii|\d+)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 5. Standardize whitespace: Replace multiple whitespaces (including newlines) with single space
    # This step is crucial, but we'll re-introduce structured newlines in advanced cleaning.
    text = re.sub(r'\s+', ' ', text).strip()
    # The following two lines were trying to re-introduce newlines based on sentences.
    # For initial cleaning, it's better to just flatten to single spaces. We'll handle
    # paragraph reconstruction in advanced cleaning.
    # text = re.sub(r'([.?!])\s*(?=[A-Z0-9]|$)', r'\1\n\n', text)
    # text = re.sub(r'\n{3,}', '\n\n', text)

    # 6. Remove hyphenation from words split across lines
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)

    # 7. Clean specific table artifacts from contents that might be extracted oddly
    text = re.sub(r'"([^"]+)"\s*,\s*"([^"]+)"', r'\1 \2', text)
    text = re.sub(r'Objectives and Scope\n', '', text)
    text = re.sub(r'Table \d+\.\d+:.*?\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'The following table:', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Mode LastWriteTime Length ---- ------------- ------', '', text)

    return text

# --------------------------------------------------------------------------------------------------
## Advanced Text Cleaning Function
# --------------------------------------------------------------------------------------------------

def advanced_text_cleaning(text, file_path=""):
    """
    Performs advanced cleaning on initially cleaned text from PDFs.
    This phase focuses on structural cleanup, boilerplate removal, and final formatting.

    Args:
        text (str): The initially cleaned text extracted from a PDF.
        file_path (str): The original file path, useful for context or logging.

    Returns:
        str: The comprehensively cleaned text.
    """
    print(f"  Starting advanced cleaning for: {os.path.basename(file_path)}")

    # 1. Normalize line breaks and remove excessive spaces within lines
    # Convert all newline variations to a single newline, then remove multiple spaces
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    text = re.sub(r' {2,}', ' ', text) # Replace multiple spaces with a single space

    # 2. Re-handle hyphenation at line breaks
    # Look for a hyphen followed by a space/newline and then a word character. Join them.
    # This specifically targets words broken at line endings.
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    # Also handle cases where a hyphen might be left with a space before a newline.
    text = re.sub(r'(\w+)\s*-\n(\w+)', r'\1\2', text)


    # 3. Aggressive boilerplate and repetitive element removal (refine based on observations)
    # These patterns are specifically crafted based on the samples you provided.
    # You will likely need to *add more* as you review your full cleaned output.
    boilerplate_patterns = [
        # Copyrights, ISBNs, Publication info
        r'Copyright\s*©.*?\d{4}.*?All\s*rights\s*reserved\.?',
        r'ISBN\s*\(\d+\):\s*\S+',
        r'ISBN\s*\(\d+\):\s*\S+.*?(?:\s*\(electronic\s*version\)|print\s*version)',
        r'This\s*edition\s*first\s*published\s*\d{4}.*?Cambridge\s*Scholars\s*Publishing',
        r'British\s*Library\s*Cataloguing\s*in\s*Publication\s*Data.*?',
        r'No\s*part\s*of\s*this\s*book\s*may\s*be\s*reproduced,\s*stored\s*in\s*a\s*retrieval\s*system,.*?(?:copyright\s*owner|Copyright\s*Licensing\s*Agency|M&K\s*Publishing)',
        r'Copyright\s*\d{4}\s*Wolters\s*Kluwer\.?', # For Wallach
        r'Prepress\s*Vendor:\s*\S+\s*\S+', # For Wallach

        # WHO/NICE Guideline specific headers/footers/disclaimers
        r'www\.nice\.org\.uk/guidance/\S+',
        r'Subject\s*t\s*o\s*Notice\s*of\s*right\s*s\s*\(https://www\.nice\.org\.uk/terms-and-\s*conditions#notice-of-rights\)\.?',
        r'Your\s*responsi\s*bility\s*The\s*recommendations\s*in\s*t\s*his\s*guideline\s*represent\s*t\s*he\s*view\s*of\s*NICE.*?(?:\.org\.uk/t\s*erms-and-\s*conditions#notice-of\s*-right\s*s)\)\.', # More robust NICE disclaimer
        r'WHO\s*GUIDELINES\s*ON\s*PHYSICAL\s*ACTIVITY\s*AND\s*SEDENTARY\s*BEHAVIOUR',
        r'WHO\s*guidelines\s*on\s*physical\s*activity\s*and\s*sedentary\s*behaviour\s*ISBN.*?(?:print\s*edition)\)',
        r'This\s*work\s*is\s*available\s*under\s*the\s*Creative\s*Commons\s*Attribution-NonCommercial-\s*ShareAlike\s*\d+\.\d+\s*IGO\s*licence.*?(?:Creative\s*Commons\s*licence\.)',
        r'In\s*any\s*use\s*of\s*this\s*work,\s*there\s*should\s*be\s*no\s*suggestion\s*that\s*WHO\s*endorses\s*any\s*specific\s*organization,\s*products\s*or\s*services\.',
        r'The\s*use\s*of\s*the\s*WHO\s*logo\s*is\s*not\s*permitted\.',
        r'This\s*is\s*an\s*advanced\s*proof,\s*which\s*is\s*being\s*made\s*available\s*for\s*information\s*only\.',

        # E-book/Platform Specific Watermarks and Footers
        r'Freemrdicalbooks\d*download', # From CURRENT Medical Diagnosis and Treatment
        r'Document\s*generated\s*by\s*Anna’s\s*Archive.*?(?:\.html)\)', # From Robbins
        r'Images\s*have\s*been\s*losslessly\s*embedded\.',
        r'Information\s*about\s*the\s*original\s*file\s*can\s*be\s*found\s*in\s*PDF\s*attachments\.',
        r'Some\s*stats\s*\(more\s*in\s*the\s*PDF\s*attachments\):.*?crc32:\s*\d+,',
        r'Thank\s*you\s*for\s*purchasing\s*this\s*e-book\..*?Sign\s*Up', # From Rubin's
        r'Access\s*to,\s*and\s*online\s*use\s*of,\s*content\s*through\s*the\s*Student\s*Consult\s*website\s*is\s*for\s*individual\s*use\s*only;',
        r'For\s*technical\s*assistance:\s*email\s*\S+@\S+\s*call\s*\S+\s*\(inside\s*the\s*US\)\s*/\s*call\s*\S+\s*\(outside\s*the\s*US\)',
        r'Important\s*note:\s*Purchase\s*of\s*this\s*product\s*includes\s*access\s*to\s*the\s*online\s*version\s*of\s*this\s*edition\s*for\s*use\s*exclusively\s*by\s*the\s*individual\s*purchaser.*?(?:will\s*invalidate\s*the\s*password\.)',
        r'Access\s*may\s*not\s*be\s*shared,\s*resold,\s*or\s*otherwise\s*circulated,\s*and\s*will\s*terminate\s*\d+\s*months\s*after\s*publication\s*of\s*the\s*next\s*edition\s*of\s*this\s*product\.',
        r'Full\s*details\s*and\s*terms\s*of',
        r'Any\s*screen\.\s*Any\s*time\s*\.\s*Anywhere\s*\.\s*Activate\s*the\s*eBook\s*version\s*of\s*this\s*title\s*at\s*no\s*additional\s*charge\s*\.\s*Unlock\s*your\s*eBook\s*today\s*\.\s*\d+\.\s*\d+\.\s*Scratch\s*box\s*below\s*to\s*reveal\s*your\s*code\s*\d+\.\s*Type\s*code\s*into\s*“Enter\s*Code\s*”\s*box\s*\d+\.\s*Click\s*“R\s*edeem\s*”\s*\d+\.\s*Log\s*in\s*or\s*Sign\s*up\s*\d+\.\s*Go\s*to\s*“M\s*y\s*Library”\s*It’s\s*that\s*easy!',
        r'Elsevier\s*eBooks\s*for\s*M\s*edical\s*Education\s*gives\s*you\s*the\s*power\s*to\s*browse\s*and\s*search',
        r'Place\s*Peel\s*Off\s*Sticker\s*HereVisit',
        r'https://t\.me/mebooksfree', # From Cecil
        r'View\s*more\s*Netter\s*titles\s*and\s*other\s*great\s*Elsevier\s*resources\s*at\s*www\.elsevierhealth\.com!', # From Netter
        r'Gain\s*a\s*better\s*view\s*of\s*medicine!.*?\.',
        r'Want\s*Netter\s*images\?.*?Netterimages\.com!',
        r'These\s*state-of-the-art\s*resources\s*from\s*Elsevier\s*complement.*?\.',
        r'For\s*Elsevier\s*Senior\s*Content\s*Strategist:.*?Illustration\s*Manager:\s*\S+\s*Rose', # From Kumar & Clark
        r'Illustrators:\s*\S+,\s*\S+',
        r'Kumar\s*&\s*Clark’s\s*Kumar\s*&\s*Clark’s\s*Clinical\s*Medicine\s*Eighth\s*Edition\s*Edited\s*by\s*Professor\s*Parveen\s*Kumar.*?(?:Toronto\s*\d{4})',
        r'About\s*this\s*release\s*FRONT\s*MATTER\s*PART\s*ONE\s*-\s*INTRODUCTION\s*TO\s*CLINICAL\s*MEDICINE.*?(?:PalmWarez\s*This\s*is\s*the\s*complete\s*text\s*of\s*Harrison\'s\s*Principles\s*of\s*Medic)', # From Harrison's B1
        r'Mandell,\s*Douglas,\s*and\s*Bennett’s\s*Principles\s*and\s*Practice\s*of\s*INFECTIOUS\s*DISEASES.*?(?:New\s*York\s*\d{5}-\d{4})', # From Mandell
        r'A-Z\s*Index\s*Librarians\s*Newsletter\s*Subscriptions\s*About\s*Healthcare\s*Jobs\s*by\s*\( Log\s*Out\s*\)\s*SEARCH\s*All\s*AccessMedicine\s*Enable\s*Autosuggest', # From toaz.info
        r'Features\s*LANGE\s*Case\s*Files\s*TM\s*Apply\s*your\s*knowledge\s*to\s*life\s*clinical\s*cases!',
        r'Prepare\s*for\s*tough\s*questions\s*on\s*rotations\s*and\s*ace\s*exams\.',
        r'View\s*CasesChapter\s*\d+\s*Developmental\s*Psychology\s*Chapter\s*\d+\s*Psychopharmacologic\s*Interventions\s*Chapter\s*\d+\s*Behavioral\s*and\s*Cognitive–\s*Behavioral\s*Interventions\s*Chapter\s*\d+\s*Psychodynamic\s*and\s*Social\s*Intervent',
        r'GR\s*AY’S\s*ANATOMY\s*FOR\s*STUDENTS\s*Fourth\s*Edition\s*Richard\s*L\.\s*Drake,\s*PhD,\s*FAAA.*?(?:Ansell\s*Horn)', # From Gray's Anatomy
        r'GRAY’S\s*ANATOMY\s*FOR\s*STUDENTS,\s*FOURTH\s*EDITION\s*ISBN:.*?(?:from\s*the\s*Elsevier\s*Inc\.)',
        r'ACCESS\s*CODE\s*INSIDE\s*ADVANTAGE\s*ACCESS\s*"\s*\'\s*Table\s*of\s*Contents\s*SECTION\s*ONE\s*General\s*Orthopaedics.*?(?:Therapies\s*for\s*Osteoarthritis)', # From Essentials of Musculoskeletal Care
        r'OXFORD\s*MEDICAL\s*PUBLICATIONS\s*Oxford\s*Handbook\s*of\s*General\s*Practice\s*ALGRAWANYii\s*Published\s*and\s*forthcoming\s*Oxford\s*Handbooks.*?(?:Dental\s*Patient\s*Care)', # From Oxford Handbook
        r'Saturated\s*fatty\s*acid\s*and\s*trans\s*-fatty\s*acid\s*intake\s*for\s*adults\s*and\s*children\s*WHO\s*guideline', # From fatty acid intake
        r'Saturated\s*fatty\s*acid\s*and\s*trans-fatty\s*acid\s*intake\s*for\s*adults\s*and\s*children:\s*WHO\s*guideline\s*ISBN.*?(?:print\s*version)\)',
        r'©\s*World\s*Health\s*Organization\s*\d{4}\s*Some\s*rights\s*reserved\.',
        r'This\s*document\s*does\s*not\s*r\s*epresent\s*f\s*ormal\s*NICE\s*guidance\.\s*F\s*or\s*a\s*full\s*list\s*of\s*NICE\s*indicat\s*ors,\s*see\s*our\s*menu\s*of\s*indicat\s*ors\.\s*To\s*find\s*out\s*ho\s*w\s*to\s*use\s*indicat\s*ors\s*and\s*ho\s*w\s*we\s*develop\s*t\s*hem,\s*see\s*our\s*NICE\s*indicat\s*or\s*process\s*guide\s*\.',
        r'Rationale\s*Monit\s*oring\s*BMI\s*can\s*help\s*identify\s*w\s*eight\s*gain\s*in\s*childr\s*en\s*and\s*y\s*oung\s*people\s*wit\s*h\s*diabet\s*es\s*so\s*appr\s*opriat\s*e\s*action\s*can\s*be',
    ]

    for pattern in boilerplate_patterns:
        text = re.sub(pattern, ' ', text, flags=re.IGNORECASE | re.DOTALL | re.MULTILINE)

    # 4. Remove standalone single characters, especially at the start/end of lines,
    # which might be artifacts from columns or formatting.
    # Be careful not to remove legitimate single-letter words (e.g., "a", "I").
    # This targets single non-alphanumeric characters or isolated letters often found in PDF artifacts.
    text = re.sub(r'(\s|^)[^A-Za-z0-9\s](\s|$)', r'\1\2', text)
    text = re.sub(r'\s+([a-z])\s+\n', r' \1\n', text) # Handle single lowercase letters before newline

    # 5. Remove non-ASCII characters that weren't caught by (cid:xx) or other initial cleaning.
    # This might remove some legitimate foreign characters if your content isn't strictly English ASCII.
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    # 6. Standardize punctuation and symbols that might have variations
    text = text.replace('—', ' -- ').replace('–', ' - ').replace('…', '...') # Standardize dashes, ellipses
    text = re.sub(r'\[PAGE_TEXT_EXTRACTION_ERROR\]', ' ', text) # Replace page error markers

    # 7. Final pass to standardize whitespace and paragraph breaks
    # Replace multiple newlines with a standard paragraph break (two newlines)
    text = re.sub(r'\n{3,}', '\n\n', text)
    # Replace any remaining multiple spaces with a single space
    text = re.sub(r' {2,}', ' ', text)
    # Remove leading/trailing whitespace from the entire text
    text = text.strip()

    print(f"  Advanced cleaning complete for: {os.path.basename(file_path)}. Cleaned length: {len(text)} characters.")
    return text

# --------------------------------------------------------------------------------------------------
## Main Processing Loop (Modified)
# --------------------------------------------------------------------------------------------------

# Dictionary to store the advanced cleaned texts
advanced_cleaned_texts_by_book = {}

for pdf_file_path in pdf_files: # Renamed variable for clarity (it's already a full path)
    # No need for os.path.join if pdf_files already contain full paths
    print(f"\n--- Processing: {os.path.basename(pdf_file_path)} ---")

    raw_text = extract_text_from_pdf(pdf_file_path)
    if raw_text:
        initial_cleaned_text = clean_initial_pdf_text(raw_text)
        print(f"Initial cleaning complete for {os.path.basename(pdf_file_path)}. Cleaned length: {len(initial_cleaned_text)} characters.")

        # --- Call the advanced cleaning phase here ---
        final_cleaned_text = advanced_text_cleaning(initial_cleaned_text, pdf_file_path)
        advanced_cleaned_texts_by_book[pdf_file_path] = final_cleaned_text
        # Optional: Save the final cleaned text to a file or database if needed
    else:
        print(f"Skipping {os.path.basename(pdf_file_path)} due to extraction error or empty content.")

print("\n--- Final Advanced Text Cleaning Summary ---")
for book_path, text_content in advanced_cleaned_texts_by_book.items():
    print(f"Book: {os.path.basename(book_path)[:35]}... | Length: {len(text_content)} chars")
    # Print the first 1000 characters of the *advanced* cleaned text
    print("Sample (first 1000 chars):\n", text_content[:1000])
    print("-" * 50)


--- Processing: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf ---
  Extracting text from 172 pages...
Successfully extracted raw text from: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
Initial cleaning complete for ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Cleaned length: 261476 characters.
  Starting advanced cleaning for: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
  Advanced cleaning complete for: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Cleaned length: 260158 characters.

--- Processing: Wallach.pdf ---
  Extracting text from 1884 pages...
Successfully extracted raw text from: Wallach.pdf
Initial cleaning complete for Wallach.pdf. Cleaned length: 3219159 characters.
  Starting advanced cleaning for: Wallach.pdf
  Advanced cleaning complete for: Wallach.pdf. Cleaned lengt

In [6]:
!pip install torch sentence-transformers

In [7]:
pip install langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Ayush Jindal\Desktop\ChatBotProject\chatbot_env\Scripts\python.exe -m pip install --upgrade pip


In [10]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\Ayush Jindal\Desktop\ChatBotProject\chatbot_env\Scripts\python.exe -m pip install --upgrade pip


In [8]:
import PyPDF2
import re
import os
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import torch # Ensure torch is imported, as it's a dependency of sentence_transformers

# --- Configuration ---
pdf_dir = 'books' # Your PDF directory

pdf_files = [
    'C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf',
    'C:/Users/Ayush Jindal/Downloads/Wallach.pdf',
    'C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf',
    "C:/Users/Ayush Jindal/Downloads/guidelines-on-mental-health-promotive-and-preventive-interventions-for-adolescents-hat_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/overweight-and-obesity-management-pdf-66143959958725_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/constipation-in-children-and-young-people-diagnosis-and-management-pdf-975757753285_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-in-adults-prevention-and-lifestyle-weight-management-programmes-pdf-75545293071301_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/fatty acid intake.pdf",
    "C:/Users/Ayush Jindal/Downloads/WHO guideline for physics activity.pdf",
    "C:/Users/Ayush Jindal/Downloads/physical-activity-exercise-referral-schemes-pdf-1996418406085_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/bipolar-disorder-assessment-and-management-pdf-35109814379461_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/obesity-clinical-assessment-and-management-pdf-75545363615173_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/IND301-20250415_copy.pdf",
    "C:/Users/Ayush Jindal/Downloads/hypertension.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICE 1.pdf",
    "C:/Users/Ayush Jindal/Downloads/NICD guideline mental health.pdf",
    "C:/Users/Ayush Jindal/Downloads/adolescent-health.pdf",
    "C:/Users/Ayush Jindal/Downloads/B1.pdf",
    "C:/Users/Ayush Jindal/Downloads/Mandell, Douglas, and Bennetts Principles and Practice of Infectious Diseases 2 Vol. Set, Eighth Edition (John E. Bennett, Raphael Dolin etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Kumar and Clarks Clinical Medicine (Parveen Kumar, Michael L Clark) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Harrisons Principles of Internal Medicine, Twenty-First Edition (Vol.1  Vol.2) (Joseph Loscalzo, Anthony S. Fauci etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Robbins pathologic basis of disease. (Cotran, Ramzi S., 1932-,Saunders etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Rubin's Pathology Clinicopathologic Foundations of Medicine, 7th Edition.pdf",
    "C:/Users/Ayush Jindal/Downloads/Netters sports medicine (Madden Ch.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Oxford Handbook of General Practice ( etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/toaz.info-current-diagnosis-amp-treatment-psychiatrypdf-pr_3071492a6593c0055faf81e303800df3.pdf",
    "C:/Users/Ayush Jindal/Downloads/Grays Anatomy For Students 4th-edition.pdf",
    "C:/Users/Ayush Jindal/Downloads/Essentials of Musculoskeletal Care, 5th ed (April Armstrong, Mark C Hubbard etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Epidemiology (Leon Gordis) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/CURRENT Medical Diagnosis and Treatment 2021 Maxine A. Papadakis, Stephen J. McPhee, Michael W. Rabow, ( etc.) (Z-Library).pdf",
    "C:/Users/Ayush Jindal/Downloads/Cecil Essentials of Medicine (Edward J. Wing MD  FACP  FIDSA (editor) etc.) (Z-Library).pdf"
]

# --- Text Extraction Function ---
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                except Exception as page_e:
                    text += "[PAGE_TEXT_EXTRACTION_ERROR]"
    except FileNotFoundError:
        print(f"Error: PDF file not found at '{pdf_path}'. Please check the path and filename.")
    except Exception as e:
        print(f"Error opening or reading PDF '{pdf_path}': {e}")
    return text

# --- Initial Text Cleaning Function ---
def clean_initial_pdf_text(text):
    text = re.sub(r'\(cid:\d+\)', '', text)
    text = re.sub(r'Routine Blood\n Results\n Explained\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Routine Blood Results Explained', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in\n Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'A practical guide\n to interpreting blood\n test results', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^\s*\d+\s*(\n|$)', '\n', text, flags=re.MULTILINE)
    text = re.sub(r'(\n|^)\s*\d+\s*(\n|$)', '\n', text)
    text = re.sub(r'Page \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Chapter \d+\s*\d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^\s*\d{1,3}\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*(viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r'^\s*(Contents|Preface|Abbreviations|Introduction|Part \d+|Chapter \d+).*?\b(?:vii|viii|ix|x|xi|xii|\d+)\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    text = re.sub(r'"([^"]+)"\s*,\s*"([^"]+)"', r'\1 \2', text)
    text = re.sub(r'Objectives and Scope\n', '', text)
    text = re.sub(r'Table \d+\.\d+:.*?\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'The following table:', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Mode LastWriteTime Length ---- ------------- ------', '', text)
    return text

# --------------------------------------------------------------------------------------------------
## Advanced Text Cleaning Function (Problematic Patterns REMOVED)
# --------------------------------------------------------------------------------------------------

def advanced_text_cleaning(text, file_path=""):
    print(f"  Starting advanced cleaning for: {os.path.basename(file_path)}")

    text = text.replace('\r\n', '\n').replace('\r', '\n')
    text = re.sub(r' {2,}', ' ', text)

    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    text = re.sub(r'(\w+)\s*-\n(\w+)', r'\1\2', text)

    # --- SIMPLIFIED BOILERPLATE PATTERNS ---
    # Several extremely long and complex patterns that caused "unbalanced parenthesis" errors
    # have been REMOVED to ensure error-free execution.
    # This might mean some complex TOCs or long disclaimers are not fully removed.
    # You can later add simpler, targeted patterns if needed.
    boilerplate_patterns = [
        # --- GENERAL COPYRIGHTS, ISBNS, PUBLICATION INFO ---
        r'Copyright\s*©.*?\d{4}.*?All\s*rights\s*reserved\.?',
        r'ISBN\s*\(?\d+\)?:\s*\S+',
        r'ISBN\s*\(?\d+\)?:\s*\S+.*?(?:\s*\(electronic\s*version\)|print\s*version\))',
        r'This\s*edition\s*first\s*published\s*\d{4}.*?Cambridge\s*Scholars\s*Publishing',
        r'British\s*Library\s*Cataloguing\s*in\s*Publication\s*Data.*?(?:British\s*Library|Copyright\s*owner)\.?',
        r'No\s*part\s*of\s*this\s*book\s*may\s*be\s*reproduced(?:,\s*stored\s*in\s*a\s*retrieval\s*system,?)?(?:,\s*or\s*transmitted,?)?(?: in any form or by any means, electronic, mechanical, photocopying(?: ,)? recording or otherwise,?)?(?: without the prior permission of the copyright owner| without either the prior permission of the publishers or a licence permitting restricted copying in the United Kingdom issued by the Copyright Licensing Agency,.*?)',
        r'Copyright\s*\d{4}\s*Wolters\s*Kluwer\.?', # For Wallach
        r'Prepress\s*Vendor:\s*\S+\s*\S+', # For Wallach
        r'This\s*book\s*is\s*protected\s*by\s*copyright\.\s*No\s*part\s*of\s*this\s*book\s*may\s*be\s*reproduced\s*or\s*transmitted\s*in\s*any\s*form\s*or\s*by\s*any\s*means,\s*including\s*as\s*photocopies\s*or\s*scanned-in\s*or', # Wallach specific copyright part
        r'For\s*the\s*full\s*range\s*of\s*M&K\s*Publishing\s*books\s*please\s*visit\s*our\s*website:\s*ww\.mkupdate\.co\.uk', # Blood Results in Clinical Practice
        r'Permissions\s*may\s*be\s*sought\s*directly\s*from\s*M&K\s*Publishing,\s*phone:\s*\d+\s*\d+,\s*fax:\s*\d+\s*\d+\s*or\s*email:\s*\S+@\S+\s*Any\s*person\s*who\s*does\s*any\s*unauthorised\s*act\s*in', # Blood Results in Clinical Practice
        r'All\s*rights\s*reserved\.\s*No\s*part\s*of\s*this\s*publication\s*may\s*be\s*reproduced,\s*stored\s*in\s*a\s*retrieval\s*system,\s*or\s*transmitted\s*in\s*any\s*form\s*or\s*by\s*any\s*means,\s*electronic,\s*mechanical,\s*photocopying,\s*recording\s*or\s*otherwise,\s*without\s*either\s*the\s*prior\s*permission\s*of\s*the\s*publishers\s*or\s*a\s*licence\s*permitting\s*restricted\s*copying\s*in\s*the\s*United\s*Kingdom\s*issued\s*by\s*the\s*Copyright\s*Licensing\s*Agency,\s*\d+\s*Tottenham\s*Court\s*Road,\s*London,\s*W\d+T\s*\d+LP\.',
        r'Details\s*on\s*how\s*to\s*seek\s*permission,\s*further\s*information\s*about\s*the\s*Publisher\s*s\s*permissions\s*policies,\s*and\s*our\s*arrangements\s*with\s*organizations\s*such\s*as\s*the\s*Copyright\s*Clearance\s*Center\s*and\s*the\s*Copyright\s*Licensing\s*Agency\s*can\s*be\s*found\s*at\s*our\s*website:\s*www\.elsevier\.com/permissions\s*This\s*book\s*and\s*the\s*individual\s*contributions\s*contained\s*in\s*it\s*are\s*protected\s*under\s*copyright\s*by\s*the\s*Publisher\s*\(other\s*than\s*as\s*may\s*be\s*noted\s*herein\)\.',

        # --- WHO/NICE GUIdeline specific headers/footers/disclaimers ---
        r'www\.nice\.or\s*g\.uk/guidance/\S+',
        r'Subject\s*t\s*o\s*Notice\s*of\s*right\s*s\s*\(https://www\s*\.nice\.or\s*g\.uk/t\s*erms-and-\s*conditions#notice-of\s*-right\s*s\)\.?',
        r'Your\s*r\s*esponsi\s*bility\s*The\s*r\s*ecommendations\s*in\s*t\s*his\s*guideline\s*r\s*epresent\s*t\s*he\s*view\s*of\s*NICE,\s*arriv\s*ed\s*at\s*aft\s*er\s*car\s*e\s*ful\s*consideration\s*of\s*t\s*he\s*evidence\s*a\s*vailable\.\s*When\s*e\s*xercising\s*t\s*heir\s*judgement,\s*pr\s*ofessionals\s*and\s*practitioners\s*ar\s*e\s*expect\s*ed\s*to\s*tak\s*e\s*this\s*guideline\s*fully\s*int\s*o\s*account,\s*alongside\s*t\s*he\s*individual\s*needs,\s*pr\s*eferences\s*and\s*v\s*alues\s*of\s*t\s*heir\s*patient\s*s\s*or\s*t\s*he\s*people\s*using\s*t\s*heir\s*ser\s*vice\.\s*It\s*is\s*not\s*mandat\s*ory\s*to\s*apply\s*t\s*he\s*recommendations,\s*and\s*t\s*he\s*guideline\s*does\s*not\s*o\s*verride\s*the\s*responsibility\s*t\s*o\s*mak\s*e\s*decisions\s*appr\s*opriat\s*e\s*to\s*the\s*cir\s*cumstances\s*of\s*t\s*he\s*individual,\s*in\s*consultation\s*wit\s*h\s*them\s*and\s*t\s*heir\s*f\s*amilies\s*and\s*car\s*ers\s*or\s*guar\s*dian\.',
        r'All\s*pr\s*oblem(?:s|\s*\(adv\s*erse\s*e\s*vents\)\s*related\s*to\s*a\s*medicine\s*or\s*medical\s*de\s*vice\s*used\s*f\s*or\s*tre)',
        r'WHO\s*GUIDELINES\s*ON\s*PHYSICAL\s*ACTIVITY\s*AND\s*SEDENTARY\s*BEHAVIOUR',
        r'WHO\s*guidelines\s*on\s*physical\s*activity\s*and\s*sedentary\s*behaviour\s*ISBN.*?(?:print\s*edition\))',
        r'This\s*is\s*an\s*advanced\s*proof,\s*which\s*is\s*being\s*made\s*available\s*for\s*information\s*only\.',
        r'The\s*designations\s*employed\s*and\s*the\s*presentation\s*of\s*the\s*material\s*in\s*this\s*publication\s*do\s*not\s*imply\s*the\s*expression\s*of\s*any\s*opinion\s*whatsoever\s*on\s*the\s*part\s*of\s*the\s*World\s*Health\s*Organization\s*concerning\s*the\s*legal\s*status\s*of\s*any\s*country,\s*territory,\s*city\s*or\s*area\s*or\s*of\s*its\s*authorities,\s*or\s*concerning\s*the',
        r'Contact\s*World\s*Health\s*Organization\s*Mental\s*Health\s*and\s*Substance\s*Use\s*Avenue\s*Appia\s*\d+\s*\d+\s*Geneva\s*\d+\s*Switzerland\s*mailto:\S+@\S+\s*www\.who\.int/teams/mental-health-and-substance-use',
        r'This\s*document\s*does\s*not\s*r\s*epresent\s*f\s*ormal\s*NICE\s*guidance\.\s*F\s*or\s*a\s*full\s*list\s*of\s*NICE\s*indicat\s*ors,\s*see\s*our\s*menu\s*of\s*indicat\s*ors\.\s*To\s*find\s*out\s*ho\s*w\s*to\s*use\s*indicat\s*ors\s*and\s*ho\s*w\s*we\s*develop\s*t\s*hem,\s*see\s*our\s*NICE\s*indicat\s*or\s*process\s*guide\s*\.',

        # --- E-BOOK/PLATFORM SPECIFIC WATERMARKS AND FOOTERS ---
        r'Freemrdicalbooks\d*download',
        r'Document\s*generated\s*by\s*Anna\s*s\s*Archive.*?(?:\.html\))\.?',
        r'Images\s*have\s*been\s*losslessly\s*embedded\.',
        r'Information\s*about\s*the\s*original\s*file\s*can\s*be\s*found\s*in\s*PDF\s*attachments\.',
        r'Some\s*stats\s*\(more\s*in\s*the\s*PDF\s*attachments\):\s*\{\s*"filename":.*?\}',
        r'Thank\s*you\s*for\s*purchasing\s*this\s*e-book\.\s*below\.\s*Or\s*visit\s*LWW\.com\s*Sign\s*Up',
        r'Use\s*of\s*the\s*current\s*edition\s*of\s*the\s*electronic\s*version\s*of\s*this\s*book\s*\(eBook\)\s*is\s*subject\s*to\s*the\s*terms\s*of\s*the\s*nontransferable,\s*limited\s*license\s*granted\s*on\s*studentconsult\.inkling\.com\.\s*Access\s*to\s*the\s*eBook\s*is\s*limited\s*to\s*the\s*first\s*individual\s*who\s*redeems\s*the\s*PIN,\s*located\s*on\s*the\s*inside\s*cover\s*of\s*this\s*book,\s*at\s*studentconsult\.inkling\.com\s*and\s*may\s*not\s*be\s*transferred\s*to\s*another\s*party\s*by\s*resale,\s*lending,\s*or\s*other\s*means\.\s*\d{4}_ME\s*For\s*technical\s*assistance:\s*email\s*studentconsult\.help@elsevier\s*\.com\s*call\s*\d+\s*-\d+-\s*\d+\s*\(inside\s*the\s*US\)\s*call\s*\+\d+\s*-\d+-\d+-\d+\s*outside\s*the\s*US\)\s*e\s*studentconsult\.inkling\.com/redeem',

        r'View\s*more\s*Netter\s*titles\s*and\s*other\s*great\s*Elsevier\s*resources\s*at\s*www\.elsevierhealth\.com!',
        r'Gain\s*a\s*better\s*view\s*of\s*medicine!.*?(?:know-how\s*you\s*need!)',
        r'Want\s*Netter\s*images\?.*?Netterimages\.com!',
        r'These\s*state-of-the-art\s*resources\s*from\s*Elsevier\s*complement.*?\.',
        r'PH\d+Visit\s*www\.elsevierhealth\.com\s*today\s*and\s*view\s*our\s*entire\s*orthopaedics\s*list!',
        r'Netter\s*s\s*Sports\s*Medicine\s*Illustrations\s*by\s*Frank\s*H\.\s*Netter,\s*MD\s*CONTRIBUTING\s*ILLUSTRATORS\s*Carlos\s*A\.\s*G\.\s*Machado,\s*MD\s*John\s*A\.\s*Craig,\s*MD\s*James\s*A\.\s*P',

        r'About\s*this\s*release\s*FRONT\s*MATTER\s*PART\s*ONE\s*-\s*INTRODUCTION\s*TO\s*CLINICAL\s*MEDICINE.*?(?:PalmWarez\s*This\s*is\s*the\s*complete\s*text\s*of\s*Harrison\'s\s*Principles\s*of\s*Medic)',
        r'ine\s*\d+th\s*Edition,\s*formatted\s*for\s*use\s*on\s*a\s*Palm\s*handheld\s*with\s*iSilo\s*\d+\.\s*In\s*order\s*to\s*minimize\s*file\s*size,\s*tables\s*and\s*figures\s*were\s*not\s*included,\s*and\s*some\s*parts\s*of\s*the\s*original\s*book\s*have\s*been\s*omitted,\s*such\s*as\s*parts\s*of\s*the\s*Front\s*Matter,\s*the\s*Bibliographies,\s*the\s*Nobel\s*Prize\s*articles,\s*Color\s*Atlases,\s*and\s*Appendices\.\s*Go\s*back\s*to\s*bookHARRISON\'S\s*PRINCIPLES\s*OF\s*INTERNAL\s*MEDICINE\s*\d+TH\s*EDITION\s*FRONT\s*MATTER\s*EDITORS\s*OF\s*PREVIOUS\s*EDITIONS\s*T\.\s*R\.\s*Harrison\s*Editor-in-Chief,\s*Editions.*?(?:R\.\s*G\.\s*Petersdo)',

        r'Mandell,\s*Douglas,\s*and\s*Bennett\s*s\s*Principles\s*and\s*Practice\s*of\s*INFECTIOUS\s*DISEASESMandell,\s*Douglas,\s*and\s*Bennett\s*s\s*Principles\s*and\s*Practice\s*of\s*INFECTIOUS\s*DISEASES\s*Eighth\s*Edition\s*Volume\s*\d+\s*JOHN\s*E\.\s*BENNETT,\s*MD,\s*MACP.*?(?:New\s*York\s*\d+\s*John\s*F\.\s*Kennedy\s*Blvd\.\s*Ste\s*\d+\s*Philadelphia,\s*PA\s*\d{5}-\d{4}\s*MANDELL,\s*DOUGLAS,\s*AND\s*BENNETT\s*S\s*PRINCIPLES\s*AND\s*PRACTICE\s*OF\s*INFECTIOUS\s*DISEASES,\s*EIGHTH\s*EDITION)',

        r'Clinical\s*Medicine\s*Eighth\s*EditionKumar\s*Clark\s*s\s*Kumar\s*Clark\s*s\s*Illustrators:\s*Richard\s*Morris,\s*Ethan\s*Danielson\s*\d{4}\s*Elsevier\s*Ltd\.\s*All\s*rights\s*reserved\.',
        r'Eighth\s*Edition\s*\d{4}\s*Seventh\s*Edition\s*\d{4}Sixth\s*Edition\s*\d{4}Fifth\s*Edition\s*\d{4}Fourth\s*Edition\s*\d{4}Third\s*Edition\s*\d{4}Second\s*Edition\s*\d{4}First\s*Edition\s*\d{4}\s*ISBN',

        r'st\s*Edition\s*HARRISON\s*S\s*P\s*R\s*I\s*N\s*C\s*I\s*P\s*L\s*E\s*S\s*O\s*F\s*I\s*N\s*T\s*E\s*R\s*N\s*A\s*L\s*M\s*E\s*D\s*I\s*C\s*I\s*N\s*E\s*LOSCALZO\s*FAUCI\s*KASPER\s*HAUSER\s*LONGO\s*V\s*O\s*L\s*U\s*M\s*E\s*JAMESONHarrison\'s\s*Principles\s*of\s*Internal\s*Medicine,\s*Twenty-First\s*Edition\s*\(Vol1\s*Vol2\)\s*\d+\.\s*Cover\s*\d+\.\s*Nav\s*\d+\.\s*Cover\s*\d+\.\s*Title\s*Page\s*\d+\.\s*Copyright\s*Page\s*\d+\.\s*Contents\s*\d+\.\s*Contributors\s*\d+\.\s*Preface\s*\d+\.\s*Harrison\s*s\s*Related\s*Resources\s*\d+\.\s*PART\s*\d+\s*The\s*Profession\s*of\s*Medicine\s*\d+\.\s*\d+\s*The\s*Practice\s*of\s*Medicine.*?(?:SECTION\s*\d+\s*Pain\s*\d+\.\s*\d+\s*Pain:\s*Pathophysiology\s*an)',

        r'Rubin\s*s\s*SEVENTH\s*EDITIONCLINICOPATHOLOGIC\s*FOUNDATIONS\s*OF\s*MEDICINEEditor\s*David\s*S\.\s*Strayer,\s*MD,\s*PhD\s*Founder\s*and\s*Contributing\s*Editor\s*Emanuel\s*Rubin,\s*MD\s*EDITOR\s*David\s*S\.\s*Strayer,\s*MD,\s*PhD\s*Professor\s*of\s*Pathology\s*Department\s*of\s*Pathology\s*and\s*Cell\s*Biology\s*Jefferson\s*Medical\s*College\s*of\s*Thomas\s*Jefferson\s*University\s*Philadelphia,\s*Pennsylvania\s*FOUNDER\s*AND\s*CONTRIBUTING\s*EDITOR\s*Emanuel\s*Rubin,\s*MD\s*Gonzalo\s*Aponte\s*Distinguished\s*Professor\s*of\s*Pathology\s*Chairman\s*Emeritus\s*of\s*the\s*Department\s*of\s*Pathology\s*and\s*Cell\s*Biology\s*Jefferson\s*Medical\s*College\s*of\s*Thomas\s*Jefferson\s*University\s*Philadelphia,\s*Pennsylvania\s*ASSOCIATE\s*EDITORS\s*PathologyRubin\s*s\s*SEVENTH\s*EDITIONCLINICOPATHOLOGIC\s*FOUNDATIONS\s*OF\s*MEDICINE\s*Jeffrey\s*E\.\s*Saffitz,\s*MD,\s*PhD\s*Mallinckrodt\s*Professor\s*of\s*Medicine\s*Harvard\s*Medical\s*School\s*Chairman,\s*Department\s*of\s*Pathology\s*Beth\s*Israel\s*Deaconess\s*Medical\s*Center\s*Boston,\s*MassachusettsAlan\s*L\.\s*Schiller,\s*MD\s*Professor\s*and\s*Chairman\s*Department\s*of\s*Pathology\s*John\s*A\.\s*Burns\s*School\s*of\s*Medicine\s*Universit',

        r'Thompson\s*Netter\s*s\s*Concise\s*Orthopaedic\s*Anatomy,\s*\d+nd\s*Edition.*?(?:James\s*A\.\s*P)',

        r'Except\s*as\s*permitted\s*under\s*the\s*United\s*States\s*Copyright\s*Act\s*of\s*\d+,\s*no\s*part\s*of\s*this\s*publication\s*may\s*be\s*reproduced\s*or\s*distributed\s*in\s*any\s*form\s*or\s*by\s*any\s*means,\s*or\s*stored\s*in\s*a\s*database\s*or\s*retrieval\s*system,\s*without\s*the\s*prior\s*written\s*permission\s*of\s*the\s*publisher\.\s*ISBN:.*?(?:To\s*contact\s*a)',

        # --- TOC REMNANTS WITH DOTS ---
        r'\.{5,}\s*\d+',
        r'CONTENTS\s*Part\s*\d+:\s*Haematology\s*and\s*Blood\s*Transfusion\s*Objectives\s*and\s*Scope\s*\.{5,}\s*\d+\s*Part\s*\d+:\s*Immunology\s*Objectives\s*and\s*Scope\s*\.{5,}\s*\d+\s*Part\s*\d+:\s*Biochemistry\s*Objectives\s*and\s*Scope\s*\.{5,}\s*\d+\s*Chapter\s*\d+:\s*Investiga',
        r'Contents\s*Quality\s*stat\s*ement\s*s\s*\.{5,}\s*\d+\s*Quality\s*stat\s*ement\s*\d+:\s*.*?\s*\.{5,}\s*\d+',

        # --- MISCELLANEOUS SPECIFIC ARTIFACTS/METADATA ---
        r'Sage\s*Academic\s*Books\s*Adolescent\s*Health:\s*A\s*Multidisciplinary\s*Approach\s*to\s*Theory\s*Research,\s*and\s*Intervention\s*For\s*the\s*most\s*optimal\s*reading\s*experience\s*we\s*recommend\s*using\s*our\s*website\.\s*https://sk\.sagepub\.com/book/mono/adolescent-health/toc\s*Author:\s*Lynn\s*Rew\s*Pub\.\s*Date:\s*\d{4}\s*Product:\s*Sage\s*Academic\s*Books\s*DOI:\s*https://doi\.org/\d+\.\d+/\d+\.\d+\s*Keywords:\s*.*?Access\s*Date:\s*July\s*\d+,\s*\d{4}\s*Publisher:\s*SAGE\s*Publications,\s*Inc\.\s*City:\s*Thousand\s*Oaks\s*Online\s*ISBN:\s*\d+-\d+-\d+-\d+-\d+\s*\d{4}\s*SAGE\s*Publications,\s*Inc\s*All\s*Rights\s*Reserved\.\s*Sage\s*\d{4}\s*by\s*Sage\s*Publications,\s*Inc\.Sage\s*Academic\s*Books\s*of\s*\d+\s*Adolescent\s*Health:\s*A\s*Multidisciplinary\s*Approach\s*to\s*Theory\s*Research,\s*and\s*InterventionFront\s*Matter\s*Dedicatio',
    ]

    for pattern in boilerplate_patterns:
        text = re.sub(pattern, ' ', text, flags=re.IGNORECASE | re.DOTALL)

    text = re.sub(r'(\s|^)[^A-Za-z0-9\s](\s|$)', r'\1\2', text)
    text = re.sub(r'\s+([a-z])\s+\n', r' \1\n', text)

    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    text = text.replace('—', ' -- ').replace('–', ' - ').replace('…', '...')
    text = re.sub(r'\[PAGE_TEXT_EXTRACTION_ERROR\]', ' ', text)

    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r' {2,}', ' ', text)
    text = text.strip()

    return text

# --- Text Chunking Function ---
def chunk_text(book_title, cleaned_text, chunk_size=1000, chunk_overlap=200):
    print(f"  Starting chunking for: {book_title}")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=[
            "\n\n",
            "\n",
            r"(?<=\.\s)\s*",
            r"(?<=\?\s)\s*",
            r"(?<=\!\s)\s*",
            " ",
            ""
        ]
    )
    chunks = text_splitter.split_text(cleaned_text)
    print(f"  Chunking complete for: {book_title}. Generated {len(chunks)} chunks.")
    return chunks

# --------------------------------------------------------------------------------------------------
## Phase: Creating Vector Embeddings and DataFrame
# --------------------------------------------------------------------------------------------------

# Initialize a Sentence Transformer model (Load once for efficiency)
print("Loading Sentence Transformer model...")
# 'all-MiniLM-L6-v2' is a good balance of size and performance for general-purpose embeddings.
# This will load the model into memory.
# Ensure torch is installed: pip install torch
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print("Model loaded.")
except Exception as e:
    print(f"Error loading SentenceTransformer model: {e}")
    print("Please ensure 'torch' and 'sentence-transformers' are installed correctly.")
    print("You might need to run: pip install torch sentence-transformers")
    # Exit or handle gracefully if model cannot be loaded
    exit()

# List to store all chunks with their metadata
all_chunks_raw_metadata = []

print("\n--- Starting PDF Processing Pipeline ---")

for pdf_file_path in pdf_files:
    book_title = os.path.basename(pdf_file_path)
    print(f"\n--- Processing: {book_title} ---")

    # --- Phase 1: Extraction ---
    raw_text = extract_text_from_pdf(pdf_file_path)
    if not raw_text:
        print(f"Skipping {book_title} due to extraction error or empty content.")
        continue

    print(f"Successfully extracted raw text from: {book_title}")

    # --- Phase 2: Initial Cleaning ---
    initial_cleaned_text = clean_initial_pdf_text(raw_text)
    print(f"Initial cleaning complete for {book_title}. Cleaned length: {len(initial_cleaned_text)} characters.")

    # --- Phase 3: Advanced Cleaning ---
    final_cleaned_text = advanced_text_cleaning(initial_cleaned_text, pdf_file_path)
    print(f"Advanced cleaning complete for: {book_title}. Cleaned length: {len(final_cleaned_text)} characters.")

    # --- Phase 4: Chunking ---
    if not final_cleaned_text.strip():
        print(f"Skipping chunking for {book_title} as its cleaned text is empty.")
        continue

    chunks_for_book = chunk_text(book_title, final_cleaned_text, chunk_size=1000, chunk_overlap=200)

    # Collect chunks with metadata
    for i, chunk in enumerate(chunks_for_book):
        all_chunks_raw_metadata.append({
            'book_name': book_title, # Renamed to book_name for consistency with desired DataFrame
            'chunk_id': f"{os.path.splitext(book_title)[0]}_chunk_{i:04d}", # Unique ID for each chunk
            'chunk_text': chunk,
            'chunk_length': len(chunk)
        })

# Create DataFrame from all collected chunks
df_book_chunks = pd.DataFrame(all_chunks_raw_metadata)

if df_book_chunks.empty:
    print("\nNo chunks were generated across all PDFs. DataFrame is empty. Cannot proceed with embeddings.")
else:
    print(f"\nCollected {len(df_book_chunks)} chunks in total. Generating embeddings...")

    # Convert the 'chunk_text' column to a list for the model's input
    chunk_texts_list = df_book_chunks['chunk_text'].tolist()

    # Generate embeddings. The model.encode method handles batching and GPU usage automatically if available.
    # The show_progress_bar=True argument will display a progress bar.
    # convert_to_tensor=False ensures the output is NumPy arrays, which is convenient for pandas.
    chunk_embeddings = model.encode(chunk_texts_list, show_progress_bar=True, convert_to_tensor=False)

    print(f"Generated embeddings with shape: {chunk_embeddings.shape}")

    # Add the embeddings as a new column to the DataFrame
    # Storing NumPy arrays directly in a DataFrame column is efficient.
    df_book_chunks['embedding'] = list(chunk_embeddings) # Convert numpy arrays in list to Python list of lists for DataFrame if needed for serialization or specific use.
                                                        # However, pandas can store numpy arrays directly. `list(chunk_embeddings)` ensures each row is a list.

    print("\n--- Final Chunks DataFrame Head with Embeddings ---")
    print(df_book_chunks.head())
    print(f"\nFinal DataFrame shape with embeddings: {df_book_chunks.shape}")

    # You can now optionally save this vectorized data to a file.
    # Parquet is highly recommended for its efficiency with large datasets and numerical arrays.
    # Make sure you have 'pyarrow' installed for this: pip install pyarrow
    try:
        # Ensure 'pyarrow' is installed: pip install pyarrow
        df_book_chunks.to_parquet('vectorized_medical_book_chunks.parquet', index=False)
        print("\nVectorized data successfully saved as 'vectorized_medical_book_chunks.parquet'")
        # Get current working directory to show full path where file is saved
        import os
        print(f"File saved to: {os.path.join(os.getcwd(), 'vectorized_medical_book_chunks.parquet')}")
    except Exception as e:
        print(f"Error saving to Parquet: {e}")
        print("Please ensure 'pyarrow' or 'fastparquet' is installed for Parquet support. You may have to install it: pip install pyarrow")
        print("Attempting to save as CSV as a fallback (embeddings will be stored as strings, which is less ideal).")
        try:
            df_book_chunks.to_csv('vectorized_medical_book_chunks.csv', index=False)
            print("Vectorized data saved as 'vectorized_medical_book_chunks.csv'")
        except Exception as csv_e:
            print(f"Error saving to CSV: {csv_e}")

print("\nProcessing complete! Your text chunks are now vectorized and saved. 🚀")
print("This DataFrame is ready for use in a Vector Database or for direct similarity search.")

C:\Users\Ayush Jindal\Desktop\ChatBotProject\chatbot_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Sentence Transformer model...
Model loaded.

--- Starting PDF Processing Pipeline ---

--- Processing: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf ---
Successfully extracted raw text from: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
Initial cleaning complete for ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Cleaned length: 261476 characters.
  Starting advanced cleaning for: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
Advanced cleaning complete for: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Cleaned length: 259198 characters.
  Starting chunking for: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
  Chunking complete for: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Generated 324 chunks.

---

Batches: 100%|███████████████████████████████████████████████████████████████████| 3826/3826 [1:42:22<00:00,  1.61s/it]


Generated embeddings with shape: (122411, 384)

--- Final Chunks DataFrame Head with Embeddings ---
                                           book_name  \
0  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
1  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
2  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
3  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
4  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   

                                            chunk_id  \
0  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
1  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
2  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
3  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   
4  ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN...   

                                          chunk_text  chunk_length  \
0  Routine Blood Results Explained Routine Blood ...           998   
1  Mu sculo-Skeletal Disease Investigation of Thy...           998   
2  professionals